In [1]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse

df = pd.read_csv('SOCASTEE_02110725.csv')
df['gaze_height'] = df['gaze_height'].interpolate(method='linear')

# Apply forward fill and backward fill as a fallback for remaining NaNs
df['gaze_height'] = df['gaze_height'].fillna(method='ffill').fillna(method='bfill')

/local_scratch/slurm.563502/ipykernel_2584983/2950438459.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['gaze_height'] = df['gaze_height'].fillna(method='ffill').fillna(method='bfill')


In [2]:
df

,Unnamed: 0.1,Unnamed: 0,DATE,gaze_height
0,0,0,2007-10-01 05:00:00+00:00,12.34
1,1,1,2007-10-01 05:15:00+00:00,12.41
2,2,2,2007-10-01 05:30:00+00:00,12.47
3,3,3,2007-10-01 05:45:00+00:00,12.52
4,4,4,2007-10-01 06:00:00+00:00,12.56
...,...,...,...,...
581812,581812,581812,2024-06-24 17:30:00+00:00,12.09
581813,581813,581813,2024-06-24 17:45:00+00:00,12.14
581814,581814,581814,2024-06-24 18:00:00+00:00,12.16
581815,581815,581815,2024-06-24 18:15:00+00:00,12.16


In [7]:
df['DATE'] = pd.to_datetime(df['DATE'], format='mixed')

for i in range(len(df)-1):
    td = (df['DATE'].iat[i+1] - df['DATE'].iat[i]).total_seconds()/3600
    if td > 1:
        print(df['DATE'].iat[i], df['DATE'].iat[i+1])

2007-11-20 20:15:00+00:00 2007-11-20 21:30:00+00:00
2008-01-19 04:45:00+00:00 2008-01-25 05:00:00+00:00
2008-02-23 08:45:00+00:00 2008-02-23 10:15:00+00:00
2008-09-17 07:30:00+00:00 2008-09-17 09:00:00+00:00
2009-03-02 04:45:00+00:00 2009-03-05 05:00:00+00:00
2012-10-01 14:30:00+00:00 2012-10-01 15:45:00+00:00
2016-10-06 04:45:00+00:00 2016-10-07 05:00:00+00:00
2016-10-15 04:45:00+00:00 2016-10-17 05:00:00+00:00
2018-04-30 18:15:00+00:00 2018-06-06 18:00:00+00:00
2018-09-21 21:00:00+00:00 2018-09-21 22:45:00+00:00


In [9]:
# fill up gaps
import pandas as pd
import numpy as np

# Assume your data is in a CSV file named 'your_data.csv'
# If it's in a different format, adjust the reading method accordingly
df = pd.read_csv('SOCASTEE_02110725.csv', parse_dates=['DATE'])

# Set the datetime column as the index
df.set_index('DATE', inplace=True)

# Sort the index to ensure it's in chronological order
df.sort_index(inplace=True)

# Create a complete time range from the start to the end date with 15-minute intervals
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15T')

# Reindex the dataframe with the full range
df_filled = df.reindex(full_range)

# Interpolate missing values
# You can choose a different method if needed (e.g., 'linear', 'cubic', 'nearest')
df_filled['gaze_height'] = df_filled['gaze_height'].interpolate(method='time')

# If you want to fill very large gaps (e.g., days) with a specific value instead of interpolating, you can do:
# max_gap = pd.Timedelta(hours=6)  # Adjust this value as needed
# mask = df_filled['gaze_height'].isna() & (df_filled.index.to_series().diff() > max_gap)
# df_filled.loc[mask, 'gaze_height'] = df_filled['gaze_height'].mean()  # Or another value of your choice

# Reset the index to make datetime a column again
df_filled.reset_index(inplace=True)
df_filled.rename(columns={'index': 'DATE'}, inplace=True)

# Save the filled dataframe to a new CSV file
df_filled.to_csv('SOCASTEE_02110725r15m.csv', index=False)

print(df_filled.head())
print(f"Original shape: {df.shape}, Filled shape: {df_filled.shape}")

/local_scratch/slurm.563502/ipykernel_2584983/1387212669.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15T')


                       DATE  Unnamed: 0.1  Unnamed: 0  gaze_height
0 2007-10-01 05:00:00+00:00           0.0         0.0        12.34
1 2007-10-01 05:15:00+00:00           1.0         1.0        12.41
2 2007-10-01 05:30:00+00:00           2.0         2.0        12.47
3 2007-10-01 05:45:00+00:00           3.0         3.0        12.52
4 2007-10-01 06:00:00+00:00           4.0         4.0        12.56
Original shape: (581817, 3), Filled shape: (586711, 4)


In [11]:
df = pd.read_csv('SOCASTEE_02110725r15m.csv')
df['DATE'] = pd.to_datetime(df['DATE'], format='mixed')

for i in range(len(df)-1):
    td = (df['DATE'].iat[i+1] - df['DATE'].iat[i]).total_seconds()/3600
    if td > 15:
        print(df['DATE'].iat[i], df['DATE'].iat[i+1])

In [15]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse

df = pd.read_csv('WAYCROSS_02226500.csv', parse_dates=['datetime'])
df['gaze_height'] = df['Gage height, feet'].interpolate(method='linear')
df['DATE'] = df['datetime']
df = df[['DATE', 'gaze_height']]

# Apply forward fill and backward fill as a fallback for remaining NaNs
df['gaze_height'] = df['gaze_height'].fillna(method='ffill').fillna(method='bfill')

# Set the datetime column as the index
df.set_index('DATE', inplace=True)

# Sort the index to ensure it's in chronological order
df.sort_index(inplace=True)

# Create a complete time range from the start to the end date with 15-minute intervals
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15T')

# Reindex the dataframe with the full range
df_filled = df.reindex(full_range)

# Interpolate missing values
# You can choose a different method if needed (e.g., 'linear', 'cubic', 'nearest')
df_filled['gaze_height'] = df_filled['gaze_height'].interpolate(method='time')

# Reset the index to make datetime a column again
df_filled.reset_index(inplace=True)
df_filled.rename(columns={'index': 'DATE'}, inplace=True)

# Save the filled dataframe to a new CSV file
df_filled.to_csv('WAYCROSS_02226500r15m.csv', columns=['DATE', 'gaze_height'], index=False)

print(df_filled.head())
print(f"Original shape: {df.shape}, Filled shape: {df_filled.shape}")

/local_scratch/slurm.563502/ipykernel_2584983/3662611427.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['gaze_height'] = df['gaze_height'].fillna(method='ffill').fillna(method='bfill')
/local_scratch/slurm.563502/ipykernel_2584983/3662611427.py:20: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15T')


                       DATE  gaze_height
0 2007-10-01 04:00:00+00:00          6.2
1 2007-10-01 04:15:00+00:00          6.2
2 2007-10-01 04:30:00+00:00          6.2
3 2007-10-01 04:45:00+00:00          6.2
4 2007-10-01 05:00:00+00:00          6.2
Original shape: (581852, 1), Filled shape: (586716, 2)


In [16]:
df

,gaze_height
DATE,
2007-10-01 04:00:00+00:00,6.20
2007-10-01 04:15:00+00:00,6.20
2007-10-01 04:30:00+00:00,6.20
2007-10-01 04:45:00+00:00,6.20
2007-10-01 05:00:00+00:00,6.20
...,...
2024-06-24 17:45:00+00:00,5.76
2024-06-24 18:00:00+00:00,5.76
2024-06-24 18:15:00+00:00,5.76


In [ ]:
df